In [1]:
# MLM 방식을 이용한 Further pre-traning 방식 구현 예제
# 참고 소스 : https://towardsdatascience.com/masked-language-modelling-with-bert-7d49793e5d2c 참조 바람
import torch
import os

from tqdm.notebook import tqdm
from transformers import BertTokenizer, BertConfig, BertForMaskedLM, BertTokenizerFast
from transformers import AdamW, get_linear_schedule_with_warmup
from myutils import GPU_info, seed_everything, mlogging

logfilepath:../log/bwdataset_2022-04-18.log
logfilepath:../log/qnadataset_2022-04-18.log


In [2]:
#======================================================================================================
# *WandB 툴을 이용하여, HyperPrameter sweeps 하는 예제임(http://wandb.ai 에 회원가입 해야 함)
#  
# == HyperPrameter sweeps 과정 ==
#
# 1. 로그인
# import wandb
# wandb.login()  # 로그인 => wandb.ai 사이트에 로그인 후, API Key 입력해야 함
#
# 2. sweep_config 설정
# sweep_config ={ 
#       'method': 'random', #grid, random
#       'metric':{
#           'name': 'val_accuracy', #loss
#           'goal': 'maximize'      # loss일때는 minimize로 설정해야함
#       },
#      'parameters':{
#           'learning_rate':{
#               'distribution': 'uniform',
#               'min' : 0,
#               'max': 7e-5
#           },
#           'batch_size':{
#               'values' : [8, 16, 32]
#           },
#           'epochs':{
#               'values' : [2, 3, 4]
#           }
#       }
#}
#
#sweep_id = wandb.sweep(sweep_config, project="bert-wb-test")
#
# 3. wandb 초기화 
# - wandb.init(config=config)
#
# 4. wandb 로그설정
# - wandb.log({"val_accuracy": total_test_correct / total_test_len})
#
# 5. wand 실행
# - wandb.agent(sweep_id, train, count=3)
#
# 6. wand 종료
# -wandb.finish()
#======================================================================================================
# 1. 로그인
#!pip install wandb -qqq
import wandb
wandb.login()  # 로그인 => wandb.ai 사이트에 로그인 후, API Key 입력해야 함

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kobongsoo (use `wandb login --relogin` to force relogin)


True

In [3]:
# 2. sweep_config 설정

sweep_config ={ 
      'method': 'random', #grid, random
      'metric':{
          'name': 'val_accuracy', #loss
          'goal': 'maximize'      # loss일때는 minimize로 설정해야함
      },
     'parameters':{
          'learning_rate':{
              'distribution': 'uniform',
              'min' :1e-5,
              'max': 7e-5
          },
          'batch_size':{
              'values' : [16, 32]
          },
          'epochs':{
              'values' : [3, 4, 5]
          }
      }
}

sweep_id = wandb.sweep(sweep_config, project="bert-wb-mlm-test-2")

Create sweep with ID: d31ha4x0
Sweep URL: https://wandb.ai/kobongsoo/bert-wb-mlm-test-2/sweeps/d31ha4x0


In [4]:
# 훈련시킬 말뭉치(사전 만들때 동일한 말뭉치 이용)
input_corpus = "../korpora/kowiki_20190620/wiki_20190620_small.txt"

# eval 말뭉치 
eval_corpus = "../korpora/kowiki_20190620/wiki_eval_test.txt"

# 기존 사전훈련된 모델
model_path = "../model/bert/bmc-fpt-wiki_20190620_mecab_false_0311-nouns-0416/"

# 기존 사전 + 추가된 사전 파일
#vocab_path="tokenizer/wiki_20190620_false_0311_speical/bmc_add_wiki_20190620_false_0311.txt"
vocab_path="../model/bert/bmc-fpt-wiki_20190620_mecab_false_0311-nouns-0416/"

# 출력
OUTPATH = '../model/bert/bmc-fpt-wiki_20190620_mecab_false_0311-nouns-0418/'

token_max_len = 128

device = GPU_info()
print(device)

#seed 설정
seed_everything(222)

#logging 설정
logger =  mlogging(loggername="wb-bert-fpt-mlm", logfilename="../log/wb-bert-fpt-mlm")

True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0
logfilepath:../log/wb-bert-fpt-mlm_2022-04-18.log


In [5]:
# tokeinzier 생성
# tokenizer 생성
# => BertTokenizer, BertTokenizerFast 둘중 사용하면됨

#tokenizer = BertTokenizer(vocab_file=vocab_path, max_len=token_max_len, do_lower_case=False)
tokenizer = BertTokenizer.from_pretrained(vocab_path, max_len=token_max_len, do_lower_case=False)
# tokenizer = BertTokenizerFast(vocab_file=vocab_file, max_len=token_max_len, do_lower_case=False)


# speical 토큰 계수 + vocab 계수 - 이미 vocab에 포함된 speical 토큰 계수(5)
vocab_size = len(tokenizer.all_special_tokens) + tokenizer.vocab_size - 5 + 1
#vocab_size = len(tokenizer.all_special_tokens) + tokenizer.vocab_size - 5
print('special_token_size: {}, tokenizer.vocab_size: {}'.format(len(tokenizer.all_special_tokens), tokenizer.vocab_size))
print('vocab_size: {}'.format(vocab_size))
print('tokenizer_len: {}'.format(len(tokenizer)))


special_token_size: 27, tokenizer.vocab_size: 167537
vocab_size: 167560
tokenizer_len: 167550


In [6]:
def load_model():
    # 모델 로딩 further pre-training 
    #config = BertConfig.from_pretrained(model_path)
    #model = BertForMaskedLM.from_pretrained(model_path, from_tf=bool(".ckpt" in model_path), config=config) 
    model = BertForMaskedLM.from_pretrained(model_path)    

    #################################################################################
    # 모델 embedding 사이즈를 tokenizer 크기 만큼 재 설정함.
    # 재설정하지 않으면, 다음과 같은 에러 발생함
    # CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)` CUDA 에러가 발생함
    #  indexSelectLargeIndex: block: [306,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
    #
    #     해당 오류는 기존 Embedding(8002, 768, padding_idx=1) 처럼 입력 vocab 사이즈가 8002인데,
    #     0~8001 사이를 초과하는 word idx 값이 들어가면 에러 발생함.
    #################################################################################
    model.resize_token_embeddings(len(tokenizer))

    model.to(device)
    
    return model

In [7]:
def build_dataset(batch_size):
    from torch.utils.data import DataLoader, RandomSampler
    import sys
    sys.path.append('..')
    from myutils import MLMDataset
  
    # 각 스페셜 tokenid를 구함
    CLStokenid = tokenizer.convert_tokens_to_ids('[CLS]')
    SEPtokenid = tokenizer.convert_tokens_to_ids('[SEP]')
    UNKtokenid = tokenizer.convert_tokens_to_ids('[UNK]')
    PADtokenid = tokenizer.convert_tokens_to_ids('[PAD]')
    MASKtokenid = tokenizer.convert_tokens_to_ids('[MASK]')
    print('CLSid:{}, SEPid:{}, UNKid:{}, PADid:{}, MASKid:{}'.format(CLStokenid, SEPtokenid, UNKtokenid, PADtokenid, MASKtokenid))

    #===============================================================================
    # 학습 dataloader 생성
    train_dataset = MLMDataset(corpus_path = input_corpus,
                               tokenizer = tokenizer, 
                               CLStokeinid = CLStokenid ,   # [CLS] 토큰 id
                               SEPtokenid = SEPtokenid ,    # [SEP] 토큰 id
                               UNKtokenid = UNKtokenid ,    # [UNK] 토큰 id
                               PADtokenid = PADtokenid,    # [PAD] 토큰 id
                               Masktokenid = MASKtokenid,   # [MASK] 토큰 id
                               max_sequence_len=token_max_len,  # max_sequence_len)
                               mlm_probability=0.15,
                               overwrite_cache=False
                              )


    # 학습 dataloader 생성
    # => tenosor로 만듬
    train_loader = DataLoader(train_dataset, 
                              batch_size=batch_size, 
                              #shuffle=True, # dataset을 섞음
                              sampler=RandomSampler(train_dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                              num_workers=3
                             )
    #===============================================================================

    #===============================================================================
    # eval dataloader 생성
    eval_dataset = MLMDataset(corpus_path = eval_corpus,
                               tokenizer = tokenizer, 
                               CLStokeinid = CLStokenid ,   # [CLS] 토큰 id
                               SEPtokenid = SEPtokenid ,    # [SEP] 토큰 id
                               UNKtokenid = UNKtokenid ,    # [UNK] 토큰 id
                               PADtokenid = PADtokenid,    # [PAD] 토큰 id
                               Masktokenid = MASKtokenid,   # [MASK] 토큰 id
                               max_sequence_len=token_max_len,  # max_sequence_len)
                               mlm_probability=0.15,
                               overwrite_cache=False
                              )


    # eval dataloader 생성
    # => tenosor로 만듬
    eval_loader = DataLoader(eval_dataset, 
                              batch_size=batch_size, 
                              #shuffle=True, # dataset을 섞음
                              sampler=RandomSampler(eval_dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                              num_workers=3
                             )
     #===============================================================================

    #print(train_dataset[0])
    
    return train_loader, eval_loader

In [12]:
import time
def save_model(model, tokenizer, OUT_PATH, config):
    
     # 현재 local 시간 얻어옴(20220414-12-20)
    tm = time.localtime(time.time())  
    tt = f"batch:{config.batch_size}-ep:{config.epochs}-lr:{config.learning_rate:.9f}-{tm.tm_mon}m{tm.tm_mday}d-{tm.tm_hour}:{tm.tm_min}"
                
    TMP_OUT_PATH = OUTPATH + tt
    
    ### 전체모델 저장
    os.makedirs(TMP_OUT_PATH, exist_ok=True)
    #torch.save(model, OUTPATH + 'pytorch_model.bin') 
    # save_pretrained 로 저장하면 config.json, pytorch_model.bin 2개의 파일이 생성됨
    model.save_pretrained(TMP_OUT_PATH)

    # tokeinizer 파일 저장(vocab)
    VOCAB_PATH = TMP_OUT_PATH
    tokenizer.save_pretrained(VOCAB_PATH)
    
    logger.info(f'==> save_model : {TMP_OUT_PATH}')
                    


In [9]:
def Train_epoch(config,
                model,
                train_loader,
                eval_loader):
    
    ##################################################
    epochs = config.epochs            # epochs
    learning_rate = config.learning_rate  # 학습률
    #p_itr = 15000           # 손실률 보여줄 step 수
    #save_steps = 50000     # 50000 step마다 모델 저장
    ##################################################

    # optimizer 적용
    optimizer = AdamW(model.parameters(), 
                     lr=learning_rate, 
                     eps=1e-8) # 0으로 나누는 것을 방지하기 위한 epsilon 값(10^-6 ~ 10^-8 사이 이값 입력합)

    # 총 훈련과정에서 반복할 스탭
    total_steps = len(train_loader)*epochs
    warmup_steps = total_steps * 0.1 #10% of train data for warm-up
    
    # 손실률 보여줄 step 수
    p_itr = int(len(train_loader)*0.1)  
    
    # step마다 모델 저장
    save_steps = int(total_steps * 0.5)
    
    # 스캐줄러 생성
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps=warmup_steps, 
                                                num_training_steps=total_steps)

    itr = 1
    total_loss = 0
    total_len = 0
    total_correct = 0
    total_test_correct = 0
    total_test_len = 0
            
    list_train_loss = []
    list_train_acc = []
    list_validation_acc = []

    model.zero_grad()# 그래디언트 초기화
    for epoch in tqdm(range(epochs)):

        # model.train() # 훈련모드로 변환
        for data in tqdm(train_loader):
            model.train() # 훈련모드로 변환
            #optimizer.zero_grad()
            model.zero_grad()# 그래디언트 초기화

            # 입력 값 설정
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            token_type_ids = data['token_type_ids'].to(device)       
            labels = data['labels'].to(device)
            #print('Labels:{}'.format(labels))

            # 모델 실행
            outputs = model(input_ids=input_ids, 
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            labels=labels)

            # 출력값 loss,logits를 outputs에서 얻어옴
            loss = outputs.loss
            logits = outputs.logits
            #print('Loss:{}, logits:{}'.format(loss, logits))

            # optimizer 과 scheduler 업데이트 시킴
            loss.backward()   # backward 구함
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   # 그래디언트 클리핑 (gradient vanishing이나 gradient exploding 방지하기 위한 기법)
            optimizer.step()  # 가중치 파라미터 업데이트(optimizer 이동)
            scheduler.step()  # 학습률 감소

            # ***further pretrain 에는 손실률 계산을 넣지 않음
            # 정확도 계산하는 부분은 no_grade 시켜서, 계산량을 줄임.

            # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
            # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
            with torch.no_grad():
                
                # 손실(loss) 계산 
                total_loss += loss.item()
                
                #===========================================
                # 정확도(Accurarcy) 계산
                pred = torch.argmax(logits, dim=2)
                tmpcorrect = pred.eq(labels)

                # 예측값 중 true인값 * attention_maks가 = 1(True)인것 중에서 True값이 합이 masked에서 알아맞춘 단어 계수임
                correct = tmpcorrect*attention_mask 
                total_correct += correct.sum().item() 

                # 단어 총 수는 attension_mask가 1(True) 인 것들의 합
                total_len += attention_mask.sum().item() 
                #=========================================   
    
                # 주기마다 test(validataion) 데이터로 평가하여 손실류 계산함.
                if itr % p_itr == 0:
                    train_loss = total_loss/p_itr
                    train_acc = total_correct/total_len
                         
                    ####################################################################
                    # 주기마다 eval(validataion) 데이터로 평가하여 손실류 계산함.
                    # 평가 시작
                    model.eval()

                    #for data in tqdm(eval_loader):
                    for data in eval_loader:
                        # 입력 값 설정
                        input_ids = data['input_ids'].to(device)
                        attention_mask = data['attention_mask'].to(device)
                        token_type_ids = data['token_type_ids'].to(device)       
                        labels = data['labels'].to(device)

                        # 손실률 계산하는 부분은 no_grade 시켜서, 계산량을 줄임.
                        # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
                        # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
                        with torch.no_grad():
                            # 모델 실행
                            outputs = model(input_ids=input_ids, 
                                            attention_mask=attention_mask,
                                            token_type_ids=token_type_ids,
                                            labels=labels)

                            # 출력값 loss,logits를 outputs에서 얻어옴
                            #loss = outputs.loss
                            logits = outputs.logits

                             #===========================================
                            # 정확도(Accurarcy) 계산
                            pred = torch.argmax(logits, dim=2)
                            tmpcorrect = pred.eq(labels)

                            # 예측값 중 true인값 * attention_maks가 = 1(True)인것 중에서 True값이 합이 masked에서 알아맞춘 단어 계수임
                            correct = tmpcorrect*attention_mask 
                            total_test_correct += correct.sum().item() 

                            # 단어 총 수는 attension_mask가 1(True) 인 것들의 합
                            total_test_len += attention_mask.sum().item() 
                            #========================================= 

                    val_acc = total_test_correct/total_test_len
                    
                    logger.info('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Train Acc: {:.4f}, Val Acc:{}'.format(epoch+1, epochs, itr, train_loss, train_acc, val_acc))
                    
                    list_train_loss.append(train_loss)
                    list_train_acc.append(train_acc)
                    list_validation_acc.append(val_acc)
                 
                    
                    # wandb 로그 기록
                    wandb.log({"Loss": train_loss,
                              "train_accuracy": train_acc,
                              "val_accuracy": val_acc
                              })
                 
                    total_loss = 0
                    total_len = 0
                    total_correct = 0
                    total_test_correct = 0
                    total_test_len = 0
                    ####################################################################

                  # 모델 저장
                if itr % save_steps == 0:
                    save_model(model, tokenizer, OUTPATH, config)
                    
            itr+=1


In [10]:
# 훈련
def train(config=None):
    # wandb 초기화
    with wandb.init(config=config):
        config = wandb.config
        
        # 데이터 로더 생성
        train_loader, eval_loader = build_dataset(config.batch_size)
        
        # 모델 로딩
        tmodel = load_model()
        
        # 훈련 시작 
        Train_epoch(config, tmodel, train_loader, eval_loader)
        
        # 모델 저장
        #save_model(tmodel, tokenizer, OUTPATH, config)

In [13]:
# 훈련을 시작 (총 3번(epoch이 아님))
wandb.agent(sweep_id, train, count=2)

wandb: Agent Starting Run: 6g0qns8c with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	learning_rate: 4.151229402443651e-06
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


2022-04-18 13:15:24,533 - bwpdataset - INFO - ==>[Start] cached file read: ../korpora/kowiki_20190620/cached_lm_BertTokenizer_128_wiki_20190620_small.txt


CLSid:101, SEPid:102, UNKid:100, PADid:0, MASKid:103
*corpus:../korpora/kowiki_20190620/wiki_20190620_small.txt
*max_sequence_len:128
*mlm_probability:0.15
*CLStokenid:101, SEPtokenid:102, UNKtokenid:100, PADtokeinid:0, Masktokeid:103


2022-04-18 13:15:24,750 - bwpdataset - INFO - <==[End] Loading features from cached file ../korpora/kowiki_20190620/cached_lm_BertTokenizer_128_wiki_20190620_small.txt [took 0.215 s]


*corpus:../korpora/kowiki_20190620/wiki_eval_test.txt
*max_sequence_len:128
*mlm_probability:0.15
*CLStokenid:101, SEPtokenid:102, UNKtokenid:100, PADtokeinid:0, Masktokeid:103
*total_line: 114


...

  0%|          | 0/614 [00:00<?, ?it/s]

2022-04-18 13:27:20,080 - wb-bert-fpt-mlm - INFO - [Epoch 3/4] Iteration 1281 -> Train Loss: 0.1107, Train Acc: 0.9168, Val Acc:0.9072593540134314
2022-04-18 13:27:30,295 - wb-bert-fpt-mlm - INFO - [Epoch 3/4] Iteration 1342 -> Train Loss: 0.0982, Train Acc: 0.9241, Val Acc:0.908538535337384
2022-04-18 13:27:40,540 - wb-bert-fpt-mlm - INFO - [Epoch 3/4] Iteration 1403 -> Train Loss: 0.0988, Train Acc: 0.9258, Val Acc:0.9069395586824432
2022-04-18 13:27:50,755 - wb-bert-fpt-mlm - INFO - [Epoch 3/4] Iteration 1464 -> Train Loss: 0.1044, Train Acc: 0.9238, Val Acc:0.9094979213303486
2022-04-18 13:28:00,947 - wb-bert-fpt-mlm - INFO - [Epoch 3/4] Iteration 1525 -> Train Loss: 0.0935, Train Acc: 0.9273, Val Acc:0.9088583306683723
2022-04-18 13:28:11,156 - wb-bert-fpt-mlm - INFO - [Epoch 3/4] Iteration 1586 -> Train Loss: 0.0964, Train Acc: 0.9257, Val Acc:0.9094979213303486
2022-04-18 13:28:21,350 - wb-bert-fpt-mlm - INFO - [Epoch 3/4] Iteration 1647 -> Train Loss: 0.0962, Train Acc: 0.9259,

  0%|          | 0/614 [00:00<?, ?it/s]

2022-04-18 13:29:02,681 - wb-bert-fpt-mlm - INFO - [Epoch 4/4] Iteration 1891 -> Train Loss: 0.0826, Train Acc: 0.9358, Val Acc:0.9094979213303486
2022-04-18 13:29:12,804 - wb-bert-fpt-mlm - INFO - [Epoch 4/4] Iteration 1952 -> Train Loss: 0.0816, Train Acc: 0.9379, Val Acc:0.9094979213303486
2022-04-18 13:29:22,948 - wb-bert-fpt-mlm - INFO - [Epoch 4/4] Iteration 2013 -> Train Loss: 0.0882, Train Acc: 0.9347, Val Acc:0.9098177166613367
2022-04-18 13:29:33,043 - wb-bert-fpt-mlm - INFO - [Epoch 4/4] Iteration 2074 -> Train Loss: 0.0831, Train Acc: 0.9347, Val Acc:0.910137511992325
2022-04-18 13:29:43,254 - wb-bert-fpt-mlm - INFO - [Epoch 4/4] Iteration 2135 -> Train Loss: 0.0823, Train Acc: 0.9352, Val Acc:0.9107771026543012
2022-04-18 13:29:53,436 - wb-bert-fpt-mlm - INFO - [Epoch 4/4] Iteration 2196 -> Train Loss: 0.0803, Train Acc: 0.9367, Val Acc:0.9104573073233131
2022-04-18 13:30:03,641 - wb-bert-fpt-mlm - INFO - [Epoch 4/4] Iteration 2257 -> Train Loss: 0.0860, Train Acc: 0.9315,

Loss,█▆▅▄▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▂▂▃▃▄▄▄▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇██████████
val_accuracy,▁▂▃▄▄▅▆▆▆▆▇▇▇▇▇▇████████████████████████
Loss,0.08515
train_accuracy,0.9341
val_accuracy,0.9111
